- <b> 1. Some extra </b>
- <b> 2. Simple example with TfidfVectorizer, CountVectorizer, NearestNeighbors </b>
- <b> 3. Using scipy distances</b>
- <b> 4. Surprise library</b>
- <b> 5. Implicit ALS (AlternatingLeastSquares)</b>
- <b> 6. LightFM library</b>
- <b>  </b>

# 1. Some extra

In [ ]:
# первая практика из RS , переслушать его комменты

film_with_our_mark = []

# посчитаем нашу метрику для каждого фильма из датасета
for f in title_num_ratings.keys():
    film_with_our_mark.append(
        # средний_рейитнг_по_фильму * (текущий_рейтинг - средний_рейитнг) / (максимальный - минимальный)
        (f, title_mean_rating[f] * (title_num_ratings[f] - mean_num_ratings) / (max_num_ratings - min_num_ratings))
    )
    
    
    
film_with_new_mark = []

for f in title_num_actions.keys():
    # посчитаем нашу новую метрику для каждого фильма из датасета
    film_with_new_mark.append(
        (f, title_mean_rating[f] * (title_num_actions[f] - mean_num_ratings) / (max_num_ratings - min_num_ratings))
    )

In [ ]:
# distances. cityblock - manhattan distance; cosine - cosines distance
from scipy.spatial.distance import cityblock, cosine, euclidean, hamming, jaccard

# 2. Simple example with TfidfVectorizer, CountVectorizer, NearestNeighbors

общая идея простая - жанры/теги объединяются в единую строку, прогоняются через CountVectorizer->TfidfTransformer, далее на полученных векторах ищешь ближайших соседей NearestNeighbors

In [ ]:


# 1. функция удалит пробелы и дефисы, и полученные элементы разобъет по вертикальной линии 
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

movie_genres = movies.genres.apply(change_string).tolist()

# 2. прогон строки через CountVectorizer и TfidfTransformer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)
count_vect.vocabulary_

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
count_vect.vocabulary_


# 3. Обучение ML модели
neigh = NearestNeighbors(n_neighbors=7, n_jobs=-1, metric='euclidean')
neigh.fit(X_train_tfidf)


# 4. Создание тестовой строки и поиск рекомендаций
test = change_string("Adventure|Comedy|Fantasy|Crime")

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

res = neigh.kneighbors(X_tfidf2, return_distance=True)
movies.iloc[res[1][0]]

# 3. Using scipy distances
for creating:
- item-to-item collaborative filtering
- user-based collaborative filtering
- item-based collaborative filtering

In [ ]:
# distances. cityblock - manhattan distance; cosine - cosines distance
from scipy.spatial.distance import cityblock, cosine, euclidean, hamming, jaccard

In [ ]:
# 1. get vectors describe movies
movie_vector = {}

for movie, group in tqdm(movies_with_ratings.groupby('title')):
    movie_vector[movie] = np.zeros(num_users) #init by zeros
    
    for i in range(len(group.userId.values)):
        u = group.userId.values[i] # user index
        r = group.rating.values[i] # which rating user gave to movie
        movie_vector[movie][int(u - 1)] = r


# 2. want to know which films are similar on 'Fight Club (1999)' using user's ratings
my_fav_film = 'Fight Club (1999)'

titles = []
distances = []

for key in tqdm(movie_vector.keys()):
    
    if key == my_fav_film:
        continue
    
    titles.append(key)
    distances.append(cosine(movie_vector[my_fav_film], movie_vector[key])) 


# 3. print result
len(distances)
best_indexes = np.argsort(distances)[:10]  # индексы фильмов с наименьшей дистанцией
best_indexes

# print the max minimum films
best_movies = [(titles[i], distances[i]) for i in best_indexes]

for m in best_movies:
    print(m)

# 4. Surprise library

## https://surpriselib.com/
### install 1. conda
conda install -c conda-forge scikit-surprise

### install 2. библиотека surprise - является надстройкой над sklearn
!pip install scikit-surprise

In [1]:
from surprise import KNNWithMeans, KNNBasic # Алгоритмы для поиска ближайших соседей (должны быть также другие)
from surprise import Dataset # в dataset надо передать пандовский объект в определенном формате
from surprise import accuracy # 
from surprise import Reader # Reader - объект который считаывает датасет и превращает его в объект билиотеки surprise
from surprise.model_selection import train_test_split # 

import pandas as pd

In [2]:
# 1. Init data - preparing special data format for surprise 
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,  # user-id field (must have)
    'iid': movies_with_ratings.title,   # object (item) identifier (must have)
    'rating': movies_with_ratings.rating # interaction (must have)
})

NameError: name 'movies_with_ratings' is not defined

In [ ]:
# get max and min ratings for settings of system
min_rating = ratings.rating.min()
max_rating = ratings.rating.max()


# инициализация Ридера, куда передаем макс и минимальный рейтинг. Далее передаем в Dataset ридер и датафрейм
# init Reader
reader = Reader(rating_scale=(min_rating, max_rating))  # Reader object input includs Min and Max ratings. 
data = Dataset.load_from_df(dataset, reader)   # Dataset obj gets dataset in a cpecial format and prepared reader obj


# split to test and train
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)

In [ ]:
# 2. user_based 
# init algorithm
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',  # distance (also pearson_baseline)
    'user_based': True  #True - for User-based, False - for Item-based
})
# fit algorithm
algo.fit(trainset)

# get pred for test
test_pred = algo.test(testset)
# count rmse
accuracy.rmse(test_pred, verbose=True)

test_pred[:10] # r_ui - real rating; est - predict rating

algo.predict(uid=2, iid='Fight Club (1999)')
algo.predict(uid=2, iid='Fight Club (1999)').r_ui # real rating from user 2
algo.predict(uid=2, iid='Fight Club (1999)').est # what rating user 2 might give for the film

In [ ]:
# 3. item_based
# init algorithm
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': False  # compute similarities between items
})
# fit algorithm
algo.fit(trainset)

# get pred for test
test_pred = algo.test(testset)

# count rmse
accuracy.rmse(test_pred, verbose=True)

new_pred = algo.predict(uid=2, iid='Fight Club (1999)')
new_pred

trainset.global_mean

<b> Ready function</b>

In [ ]:
# automatic fnction
def generate_recommendation(uid, model, dataset, thresh=4, amount=5):
    all_titles = list(dataset['iid'].values)
    users_seen_titles = dataset[dataset['uid'] == uid]['iid']
    titles = np.array(list(set(all_titles) - set(users_seen_titles)))

    np.random.shuffle(titles)
    
    rec_list = []
    for title in titles:
        review_prediction = model.predict(uid=uid, iid=title)
        rating = review_prediction.est

        if rating >= thresh:
            rec_list.append((title, round(rating, 2)))
            
            if len(rec_list) >= amount:
                return rec_list

generate_recommendation(2, algo, dataset)

# 5. Implicit ALS (AlternatingLeastSquares)

In [ ]:
# implicit package - выполняет implicit als (метод AlternatingLeastSquares)
from implicit.als import AlternatingLeastSquares

# sparse matrix - разряженная матрица - метод хранения матрица у которой большая часть элементов равна нулям
import scipy.sparse as sparse

In [ ]:
## создадим разряженную матрицу sparse matrix

# проассоциируем строчки с юзерами, а столбцы с артистами
rows = data.user_id.astype(int)
cols = data.artist_id.astype(int)

# собираем sparse матрицы , куда передаем (взаимодейсвтие и (строчки, столбцы ) - это как бы индексы)
data_sparse = sparse.csr_matrix((plays, (rows, cols)), shape=(len(users),len(artists)))

## перевод в "большой вид"
userid = [0,10]
user_items = data_sparse.T.tocsr()[userid]
user_items[0][:10].todense()
np.array(items)[0][:10]
##

In [ ]:
## Init ALS model and fit modeo on sparse matrix
model = AlternatingLeastSquares(factors=50)
model.fit(data_sparse.T) # транспонирование - чтоб рекомендовать юзерам песни, а не песням юзеров  (немного контринтуитивно)


## 1. get and show recommendations (персональная для юзера)
userid = 1111
recommendations = model.recommend(userid, data_sparse[userid]) # чтоб получить рекомендацию, надо в модель передать ЮзерID и то что юзер уже слушал

for r in recommendations[0]:
    print(artist_id_name[str(r)]) # artist_id_name - dict соответсвия artist_id и artist_name


## 2. get and show recommendations (по выбронному item показать похожие)
itemid = 252494
related = model.similar_items(itemid)

for a in related[0]:
    print(artist_id_name[str(a)])

# 6. LightFM library

In [ ]:
# load a built-in dataset "fetch_movielens"
from lightfm.datasets import fetch_movielens

In [ ]:
# Init data. fetch_movielens() returns dict
movielens = fetch_movielens()

# get splitted data for constructing rs model 
train = movielens['train']
test = movielens['test']

In [ ]:
# import lightfm package
from lightfm import LightFM # basic class for fitting and predictions 
from lightfm.evaluation import precision_at_k # precision at k is a measure of how many of the top k recommended items are relevant.
from lightfm.evaluation import auc_score # calculates the Area Under the Curve score for the recommendations. The AUC is a measure of the model's ability to rank relevant items higher than irrelevant items.


# attempt 1
model = LightFM(learning_rate=0.05, loss='warp')
model.fit(train, epochs=10) # fit from "zero"
#model.fit_partial(train, epochs=10) # fit_partial - continue optimization from "reached point"


train_precision = precision_at_k(model, train, k=10).mean()

# train_interactions=train - These items won't be counted when calculating the score to prevent suggesting items the user already knows about.
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean() 

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision)) # isn't good result

In [ ]:
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        # list of positives values 
        known_positives = data['item_labels'][data['train'].tocsr()                                    
                          [user_id].indices]
        
        scores = model.predict(user_id, np.arange(n_items))

        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")
        
        for x in known_positives[:3]:
            print("        %s" % x)
        
        print("     Recommended:")
        
        for x in top_items[:3]:
            print("        %s" % x)

sample_recommendation(model, movielens, [10, 25, 451])